In [1]:
import pandas as pd
import json
import ast
import numpy as np
from sklearn.model_selection import train_test_split



## Extract parallel senetnces

In [2]:
parallel_dataset_bruk_dataset = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/bruk_parallel.csv")
parallel_dataset_ng_dataset = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/ng_parallel.csv")

In [3]:
bruk_sentence_and_ann_df = parallel_dataset_bruk_dataset[['original_sentence', 'changed_sentence']]
ng_sentence_and_ann_df = parallel_dataset_ng_dataset[['original_sentence', 'changed_sentence']]

In [4]:
ng_sentence_and_ann_df.head(20)

,original_sentence,changed_sentence
0,"Слідчі встановили, що у 2018-2019 рр. управлін...","Слідчі встановили, що у 2018-2019 рр. управлін..."
1,Суд надав слідчим доступ до документації щодо ...,Суд надав слідчій доступ до документації щодо ...
2,"Директором та засновником ТОВ «Армшляхбуд», ст...","Директоркою та засновницею ТОВ «Армшляхбуд», с..."
3,Співвласником цієї компанії у свою чергу з час...,Співвласницею цієї компанії у свою чергу з час...
4,Одесит Іван Плачков почав працювати в компанії...,Одеситка Іванна Плачкова почала працювати в ко...
5,"У 1999 очолив Міністерство енергетики України,...",У 1999 очолила Міністерство енергетики України...
6,"Його син, Плачков Григорій Іванович, з серпня ...","Її донька, Плачкова Олена Іванівна, з серпня 2..."
7,"Жовтневий райсуд м. Дніпро призначив 25,5 тис ...","Жовтневий райсуд м. Дніпро призначив 25,5 тис ..."
8,В подальшому Тютюнник пообіцяв директору ПТ «А...,В подальшому Тютюнник пообіцяв директорці ПТ «...
9,У січні цього року чиновник у салоні автомобіл...,У січні цього року чиновниця у салоні автомобі...


In [8]:
bruk_sentence_and_ann_df[35:60].to_csv("sample_of_parallel_dataset.csv", index=0)

In [47]:
parallel_df = pd.concat([ng_sentence_and_ann_df, bruk_sentence_and_ann_df])
parallel_df.reset_index(drop=True, inplace=True)

In [48]:
parallel_df.shape

(1513, 2)

In [49]:
parallel_df.head()

,original_sentence,changed_sentence
0,"Слідчі встановили, що у 2018-2019 рр. управлін...","Слідчі встановили, що у 2018-2019 рр. управлін..."
1,Суд надав слідчим доступ до документації щодо ...,Суд надав слідчій доступ до документації щодо ...
2,"Директором та засновником ТОВ «Армшляхбуд», ст...","Директоркою та засновницею ТОВ «Армшляхбуд», с..."
3,Співвласником цієї компанії у свою чергу з час...,Співвласницею цієї компанії у свою чергу з час...
4,Одесит Іван Плачков почав працювати в компанії...,Одеситка Іванна Плачкова почала працювати в ко...


In [26]:
parallel_df.to_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/parallel_df.csv", index=0)

In [50]:
parallel_df = parallel_df.rename(columns={'original_sentence': "inputs", 'changed_sentence': 'targets'})

In [51]:
parallel_df = parallel_df.sample(frac=1, random_state=42).reset_index(drop=True)

swap_mask = np.random.choice([True, False], size=len(parallel_df))  # Randomly decide which rows to swap

# Store original values to prevent overwriting columns prematurely
inputs_copy = parallel_df['inputs'].copy()
targets_copy = parallel_df['targets'].copy()

# Swap the 'inputs' and 'targets' values for the selected rows
parallel_df['inputs'] = np.where(swap_mask, targets_copy, inputs_copy)
parallel_df['targets'] = np.where(swap_mask, inputs_copy, targets_copy)

In [52]:
parallel_df.head()

,inputs,targets
0,Єдиним конкурентом на тендері було Управління ...,Єдиною конкуренткою на тендері було Управління...
1,"Це мати Ірини Скарбовійчук, дружини Руслана Ск...","Це мати Івана Скарбовійчука, чоловіка Ганни Ск..."
2,19 травня цього року вдома у директора «Аннтей...,19 травня цього року вдома у директорки «Аннте...
3,Родичі колишньої народної депутатки з фракції ...,Родичі колишнього народного депутата з фракції...
4,Однак наприкінці 2017 року директор «Харківгор...,Однак наприкінці 2017 року директорка «Харківг...


In [53]:
parallel_df['inputs'] = parallel_df.apply(lambda x: "Перефразуй це речення, змінивши гендерні сутності на протилежні (чоловічий <-> жіночий): " + x['inputs'], axis=1)

In [54]:
parallel_df.head()

,inputs,targets
0,"Перефразуй це речення, змінивши гендерні сутно...",Єдиною конкуренткою на тендері було Управління...
1,"Перефразуй це речення, змінивши гендерні сутно...","Це мати Івана Скарбовійчука, чоловіка Ганни Ск..."
2,"Перефразуй це речення, змінивши гендерні сутно...",19 травня цього року вдома у директорки «Аннте...
3,"Перефразуй це речення, змінивши гендерні сутно...",Родичі колишнього народного депутата з фракції...
4,"Перефразуй це речення, змінивши гендерні сутно...",Однак наприкінці 2017 року директорка «Харківг...


## Add word pairs (male-female)

In [2]:
gender_dict_df = pd.read_csv("/Users/linndfors/study/diploma/uk-gender-word-mapper/gender_pairs_dictionary.csv")

In [3]:
gender_dict_df = gender_dict_df.rename(columns={'male': "inputs", 'female': 'targets'})

In [ ]:
# gender_dict_df_half_1 = gender_dict_df[:len(gender_dict_df)//2]
# gender_dict_df_half_1 = gender_dict_df_half_1.rename(columns={'male': "inputs", 'female': 'targets'})

# gender_dict_df_half_2 = gender_dict_df[len(gender_dict_df)//2:]
# gender_dict_df_half_2 = gender_dict_df_half_2.rename(columns={'female': "inputs", 'male': 'targets'})
# gender_dict_df_half_2 = gender_dict_df_half_2[[gender_dict_df_half_2.columns[-1]] + list(gender_dict_df_half_2.columns[:-1])]

In [ ]:
# parallel_df_words = pd.concat([gender_dict_df_half_1, gender_dict_df_half_2])
# parallel_df_words.reset_index(drop=True, inplace=True)

In [57]:
df = gender_dict_df.sample(frac=1, random_state=42).reset_index(drop=True)

swap_mask = np.random.choice([True, False], size=len(df))  # Randomly decide which rows to swap

# Store original values to prevent overwriting columns prematurely
inputs_copy = df['inputs'].copy()
targets_copy = df['targets'].copy()

# Swap the 'inputs' and 'targets' values for the selected rows
df['inputs'] = np.where(swap_mask, targets_copy, inputs_copy)
df['targets'] = np.where(swap_mask, inputs_copy, targets_copy)

In [58]:
df

,inputs,targets
0,житель,жителька
1,памфлетист,памфлетистка
2,кар’єрист,кар’єристка
3,перевізниця,перевізник
4,диверсантка,диверсант
...,...,...
1096,корейка,кореєць
1097,вболівальник,вболівальниця
1098,хабарник,хабарниця
1099,юрист,юристка


In [60]:
df['inputs'] = df.apply(lambda x: "Перефразуй це слово, змінивши його гендер на протилежний (чоловічий <-> жіночий): " + x['inputs'], axis=1)

In [61]:
df.head()

,inputs,targets
0,"Перефразуй це слово, змінивши його гендер на п...",жителька
1,"Перефразуй це слово, змінивши його гендер на п...",памфлетистка
2,"Перефразуй це слово, змінивши його гендер на п...",кар’єристка
3,"Перефразуй це слово, змінивши його гендер на п...",перевізник
4,"Перефразуй це слово, змінивши його гендер на п...",диверсант


In [62]:
parallel_df_enhanced = pd.concat([parallel_df, df])

In [63]:
parallel_df_enhanced.head()

,inputs,targets
0,"Перефразуй це речення, змінивши гендерні сутно...",Єдиною конкуренткою на тендері було Управління...
1,"Перефразуй це речення, змінивши гендерні сутно...","Це мати Івана Скарбовійчука, чоловіка Ганни Ск..."
2,"Перефразуй це речення, змінивши гендерні сутно...",19 травня цього року вдома у директорки «Аннте...
3,"Перефразуй це речення, змінивши гендерні сутно...",Родичі колишнього народного депутата з фракції...
4,"Перефразуй це речення, змінивши гендерні сутно...",Однак наприкінці 2017 року директорка «Харківг...


In [65]:
shuffled_df = parallel_df_enhanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [70]:
shuffled_df.head(10)

,inputs,targets
0,"Перефразуй це слово, змінивши його гендер на п...",патрульний
1,"Перефразуй це речення, змінивши гендерні сутно...",Після відлучення групи Фірташа від ОПЗ постача...
2,"Перефразуй це слово, змінивши його гендер на п...",дзвонар
3,"Перефразуй це речення, змінивши гендерні сутно...",Тяганина між забудовником і депутатами загрожу...
4,"Перефразуй це слово, змінивши його гендер на п...",монтажник
5,"Перефразуй це речення, змінивши гендерні сутно...","Випивши , майстриня багато розповідала про жит..."
6,"Перефразуй це речення, змінивши гендерні сутно...",Лідія Левицька зареєструвалася як підприємець ...
7,"Перефразуй це речення, змінивши гендерні сутно...",Відповідальна редакторка — докторка філологічн...
8,"Перефразуй це речення, змінивши гендерні сутно...",Депутатка від «Самопомочі» Валентина Федишина ...
9,"Перефразуй це слово, змінивши його гендер на п...",розкрадачка


In [71]:
shuffled_df.to_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/parallel_for_llm_df.csv", index=0)

### Split

In [72]:
train_df, test_df = train_test_split(shuffled_df, test_size=0.2, random_state=42)

In [73]:
print("Train Set:")
print(len(train_df))
print("\nTest Set:")
print(len(test_df))

Train Set:
2091

Test Set:
523


In [74]:
train_df.to_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/train_parallel_for_llm_df.csv", index=0)
test_df.to_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/test_parallel_for_llm_df.csv", index=0)

In [75]:
train_df.head()

,inputs,targets
166,"Перефразуй це слово, змінивши його гендер на п...",неформалка
2576,"Перефразуй це слово, змінивши його гендер на п...",фундаторка
49,"Перефразуй це слово, змінивши його гендер на п...",абонентка
2375,"Перефразуй це речення, змінивши гендерні сутно...",Переможні битви війська Богдани Хмельницької у...
961,"Перефразуй це речення, змінивши гендерні сутно...","— Як скажеш , батьку ! — хором відповіли « пол..."


In [76]:
test_df.head()

,inputs,targets
2437,"Перефразуй це слово, змінивши його гендер на п...",офіцерка
1033,"Перефразуй це слово, змінивши його гендер на п...",мільярдер
2575,"Перефразуй це слово, змінивши його гендер на п...",пакистанець
1124,"Перефразуй це речення, змінивши гендерні сутно...",Статутні внески найбільшого розміру не задекла...
1942,"Перефразуй це слово, змінивши його гендер на п...",отоларингологиня


In [9]:
gender_dict_df

,inputs,targets
0,абітурієнт,абітурієнтка
1,абонент,абонентка
2,абориген,аборигенка
3,абстракціоніст,абстракціоністка
4,авантюрист,авантюристка
...,...,...
1096,язичник,язичниця
1097,японець,японка
1098,японіст,японістка
1099,ясновидець,ясновидиця


In [5]:
train_df = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/train_parallel_for_llm_df.csv")
test_df = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/test_parallel_for_llm_df.csv")

In [7]:
train_df['inputs'][0]

'Перефразуй це слово, змінивши його гендер на протилежний (чоловічий <-> жіночий): неформал'

In [12]:
test_df

,inputs,targets
0,"Перефразуй це слово, змінивши його гендер на п...",офіцерка
1,"Перефразуй це слово, змінивши його гендер на п...",мільярдер
2,"Перефразуй це слово, змінивши його гендер на п...",пакистанець
3,"Перефразуй це речення, змінивши гендерні сутно...",Статутні внески найбільшого розміру не задекла...
4,"Перефразуй це слово, змінивши його гендер на п...",отоларингологиня
...,...,...
518,"Перефразуй це речення, змінивши гендерні сутно...","Зазначається, що це спільна власність судді та..."
519,"Перефразуй це речення, змінивши гендерні сутно...","Львів», які перевірила декларації депутатів ЛМ..."
520,"Перефразуй це речення, змінивши гендерні сутно...",- А ти що - підглядав ? - питає міліціонер .
521,"Перефразуй це речення, змінивши гендерні сутно...",Працівник розплідника Брагіна Олександр Черняє...


In [29]:
'неформал' in list(gender_dict_df['inputs'].values)

True

In [33]:
female, male, total_words = 0, 0, 0

for x, row in train_df.iterrows():
    if "Перефразуй це слово, змінивши його гендер на протилежний (чоловічий <-> жіночий): " in row['inputs']:
        word = row['inputs'].split("Перефразуй це слово, змінивши його гендер на протилежний (чоловічий <-> жіночий): ")[1]
        total_words += 1
        if word in list(gender_dict_df['inputs'].values):
            male += 1
        elif word in list(gender_dict_df['targets'].values):
            female += 1
        else:
            print("error:", row['inputs'])
            break

In [34]:
print(female, male, total_words)

462 431 893


In [35]:
46200/893

51.73572228443449

In [36]:
43100/893

48.26427771556551

In [37]:
female, male, total_words = 0, 0, 0

for x, row in test_df.iterrows():
    if "Перефразуй це слово, змінивши його гендер на протилежний (чоловічий <-> жіночий): " in row['inputs']:
        word = row['inputs'].split("Перефразуй це слово, змінивши його гендер на протилежний (чоловічий <-> жіночий): ")[1]
        total_words += 1
        if word in list(gender_dict_df['inputs'].values):
            male += 1
        elif word in list(gender_dict_df['targets'].values):
            female += 1
        else:
            print("error:", row['inputs'])
            break

In [38]:
print(female, male, total_words)

94 114 208


In [39]:
9400/208

45.19230769230769

In [40]:
11400/208

54.80769230769231